# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/kp/Git/python-api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,46.87,99,100,19.17,TF,1743789577
1,1,st. john's,47.5649,-52.7093,30.27,92,100,16.11,CA,1743789578
2,2,hithadhoo,-0.6000,73.0833,82.72,72,91,3.51,MV,1743789579
3,3,edinburgh of the seven seas,-37.0676,-12.3116,66.29,92,32,11.77,SH,1743789580
4,4,weno,7.4515,151.8468,77.88,91,100,11.41,FM,1743789582


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size='Humidity',
    scale=0.5,
    color='City',
    colormap='RdYlBu_r',
    title='City Humidity Map',
    hover_cols=['City', 'Country', 'Humidity', 'Max Temp']
)

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values

clean_city_df = city_data_df.dropna()

my_ideal_city_df = clean_city_df[
    (clean_city_df["Max Temp"] < 85) &
    (clean_city_df["Humidity"] < 70) &
    (city_data_df["Wind Speed"] > 5) &
    (city_data_df["Wind Speed"] < 15)
]

# Display sample data

print("\nMy Ideal City")
print("~" * 50)
print(my_ideal_city_df.head())
print("\nDataset Info:")
print("*" * 50)
print(my_ideal_city_df.info())
print("\nNumber of cities in cleaned up dataset:", len(my_ideal_city_df))



My Ideal City
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    City_ID             City      Lat       Lng  Max Temp  Humidity  \
25       25           bethel  41.3712  -73.4140     66.88        45   
38       38        grytviken -54.2811  -36.5092     31.69        60   
40       40  figuig (centre)  32.1089   -1.2286     77.88        13   
44       44        subaykhan  34.8544   40.6083     73.02        31   
57       57             gobo  33.8833  135.1500     45.66        66   

    Cloudiness  Wind Speed Country        Date  
25         100        6.93      US  1743789607  
38          86        9.13      GS  1743789623  
40           0       14.58      MA  1743789626  
44          27        8.12      SY  1743789630  
57          34        7.92      JP  1743789646  

Dataset Info:
**************************************************
<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 25 to 558
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  -

/var/folders/v0/n8dxy9tj5kjf4shhfv5nfh2r0000gn/T/ipykernel_77586/2780482147.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  my_ideal_city_df = clean_city_df[


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
print("Sample of Hotel DataFrame")
print("~" * 50)
print(hotel_df.head())
print("\nDataset Info:")
print("~" *  50)
print(hotel_df.info())


Sample of Hotel DataFrame
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                          City Country      Lat       Lng  Humidity Hotel Name
0            port-aux-francais      TF -49.3500   70.2167        99           
1                   st. john's      CA  47.5649  -52.7093        92           
2                    hithadhoo      MV  -0.6000   73.0833        72           
3  edinburgh of the seven seas      SH -37.0676  -12.3116        92           
4                         weno      FM   7.4515  151.8468        91           

Dataset Info:
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'pandas.core.frame.DataFrame'>
Index: 564 entries, 0 to 567
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        564 non-null    object 
 1   Country     564 non-null    object 
 2   Lat         564 non-null    float64
 3   Lng         564 non-null    float64
 4   Humidity    564 non-null    int64

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodations.hotel",
    "apikey": geoapify_key,
    "limit": 1,
    "conditions": "radius",
    "filter": "circle",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat= row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng}, {lat}, {radius}"
    params["bias"] = f"proximity:{lng}, {lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port-aux-francais - nearest hotel: No hotel found
st. john's - nearest hotel: No hotel found
hithadhoo - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
weno - nearest hotel: No hotel found
lodja - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: No hotel found
sandnessjoen - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
vitoria da conquista - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
changji - nearest hotel: No hotel found
new kingman-butler - nearest hotel: No hotel found
luena - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
kodinsk - nearest hotel: No hotel found
kattivakkam - nearest hotel: No hotel found
maitum - nearest hotel: No hotel found
cidade velha - nearest hotel: No hotel found
rio grande - nearest hotel: No hotel found
ava

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,99,No hotel found
1,st. john's,CA,47.5649,-52.7093,92,No hotel found
2,hithadhoo,MV,-0.6000,73.0833,72,No hotel found
3,edinburgh of the seven seas,SH,-37.0676,-12.3116,92,No hotel found
4,weno,FM,7.4515,151.8468,91,No hotel found
...,...,...,...,...,...,...
563,chipinge,ZW,-20.1883,32.6236,86,No hotel found
564,boden,SE,65.8252,21.6886,87,No hotel found
565,belomorsk,RU,64.5232,34.7668,72,No hotel found
566,byron bay,AU,-28.6500,153.6167,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
city_map = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size='Humidity',
    scale=0.5,
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    title='Hotel Locations'
)

# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)